In [6]:
import os
import shutil
import requests
from bs4 import BeautifulSoup
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Function to create directory if it does not exist
def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    print(f'Directory {directory} created or already exists')

# Function to rename a file
def rename_file(old_name, new_name):
    try:
        os.rename(old_name, new_name)
        print(f'Renamed {old_name} to {new_name}')
    except Exception as e:
        print(f"An error occurred while renaming file: {e}")

# Function to move a file to a new directory
def move_file(file_name, new_directory):
    try:
        create_directory(new_directory)
        shutil.move(file_name, new_directory)
        print(f'Moved {file_name} to {new_directory}')
    except Exception as e:
        print(f"An error occurred while moving file: {e}")

# Function to scrape data from a website
def scrape_website(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        titles = [title.get_text() for title in soup.find_all('h1')]

        # Save data to CSV
        df = pd.DataFrame(titles, columns=['Title'])
        df.to_csv('scraped_data.csv', index=False)
        print('Data scraped and saved to scraped_data.csv')
        return 'scraped_data.csv'
    except Exception as e:
        print(f"An error occurred while scraping website: {e}")
        return None

# Function to process data
def process_data(file_name):
    try:
        df = pd.read_csv(file_name)
        df.dropna(inplace=True)
        summary = df.describe()
        df.to_csv('cleaned_data.csv', index=False)
        print('Data cleaned and saved to cleaned_data.csv')
        return summary
    except Exception as e:
        print(f"An error occurred while processing data: {e}")
        return None

# Function to send an email
def send_email(subject, body, recipient_email, sender_email, password, smtp_server, smtp_port):
    try:
        # Set up the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, password)

        # Create the email
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        # Send the email
        server.send_message(msg)
        server.quit()
        print('Email sent to', recipient_email)
    except Exception as e:
        print(f"An error occurred while sending email: {e}")

# Main function to orchestrate the tasks
def main():
    # Define variables
    old_name = 'example.txt'
    new_name = 'renamed_example.txt'
    new_directory = 'processed_files'
    url = 'https://example.com'
    recipient_email = 'recipient@example.com'
    sender_email = 'your_email@example.com'
    password = 'your_password'
    smtp_server = 'smtp.example.com'
    smtp_port = 587

    # Create the necessary directories
    create_directory(new_directory)

    # Perform file operations
    rename_file(old_name, new_name)
    move_file(new_name, new_directory)

    # Perform web scraping
    scraped_file = scrape_website(url)

    # Process the data if scraping was successful
    if scraped_file:
        summary = process_data(scraped_file)
        if summary is not None:
            print('Data summary:\n', summary)

            # Send an email with the results
            subject = 'Automated Task Results'
            body = f'Here is the summary of the data processed:\n\n{summary}'
            send_email(subject, body, recipient_email, sender_email, password, smtp_server, smtp_port)
        else:
            print("Data processing failed.")
    else:
        print("Web scraping failed.")

# Run the main function
if __name__ == '__main__':
    main()




Directory processed_files created or already exists
An error occurred while renaming file: [Errno 2] No such file or directory: 'example.txt' -> 'renamed_example.txt'
Directory processed_files created or already exists
An error occurred while moving file: [Errno 2] No such file or directory: 'renamed_example.txt'
Data scraped and saved to scraped_data.csv
Data cleaned and saved to cleaned_data.csv
Data summary:
                  Title
count                1
unique               1
top     Example Domain
freq                 1
An error occurred while sending email: [Errno -2] Name or service not known
